<a href="https://colab.research.google.com/github/ian293382/colab_mask_data/blob/main/mask_save_in_db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. 請問將結果 (A) 維持在 Python 程式碼執行環境中 (B) 存成一個 .csv 檔案 (C) 放入資料庫當中，這三種保存資料的方法有什麼不同？


對於需要快速訪問和處理但不需要持久存儲的數據，可以選擇保持在程式執行環境中。(即時訪問 處理)

如果數據需要被不同的使用者或程式訪問，或需要長期存儲但結構簡單，可以選擇 .csv 檔案。(易於共享和移植)

對於需要高效處理大量數據、確保數據安全和完整性的情景，使用資料庫是更好的選擇。（安全性和一致性 以及併發控制）

In [4]:
import sqlite3
import datetime
import requests
import json

# 連接至 SQLite 數據庫
conn = sqlite3.connect('mask.db')
c = conn.cursor()

# 如果不存在則創建數據表，並清空原有數據
c.execute('''
CREATE TABLE IF NOT EXISTS pharmacies (
    city TEXT,
    counts INTEGER,
    createdAt DATETIME
)''')
c.execute('DELETE FROM pharmacies')
conn.commit()

# 從 API 加載數據
url = 'https://raw.githubusercontent.com/kiang/pharmacies/master/json/points.json'
response = requests.get(url)
data = json.loads(response.text)

# 計算每個縣市的藥局數量
med_count = {}
for d in data['features']:
    county = d['properties']['county']
    if county not in med_count:
        med_count[county] = 1  # 初始化為 1
    else:
        med_count[county] += 1

# 將數據插入數據庫
for city, count in med_count.items():
    t = datetime.datetime.now()
    c.execute('INSERT INTO pharmacies (city, counts, createdAt) VALUES (?, ?, ?)', (city, count, t))
    conn.commit()

# 查詢並打印所有記錄
c.execute("SELECT * FROM pharmacies")
print(c.fetchall())

# 提交事務並關閉連接
conn.commit()
conn.close()


[('臺北市', '339', '2024-04-22 16:36:15.569339'), ('', '49', '2024-04-22 16:36:15.578410'), ('高雄市', '422', '2024-04-22 16:36:15.588737'), ('臺中市', '420', '2024-04-22 16:36:15.596703'), ('臺南市', '271', '2024-04-22 16:36:15.604511'), ('基隆市', '57', '2024-04-22 16:36:15.612288'), ('新竹市', '39', '2024-04-22 16:36:15.619951'), ('嘉義市', '67', '2024-04-22 16:36:15.627487'), ('新北市', '501', '2024-04-22 16:36:15.635167'), ('桃園市', '264', '2024-04-22 16:36:15.642586'), ('新竹縣', '46', '2024-04-22 16:36:15.650677'), ('宜蘭縣', '76', '2024-04-22 16:36:15.658540'), ('苗栗縣', '56', '2024-04-22 16:36:15.668012'), ('彰化縣', '179', '2024-04-22 16:36:15.675957'), ('南投縣', '67', '2024-04-22 16:36:15.683818'), ('雲林縣', '129', '2024-04-22 16:36:15.692021'), ('嘉義縣', '84', '2024-04-22 16:36:15.699601'), ('屏東縣', '140', '2024-04-22 16:36:15.707230'), ('澎湖縣', '11', '2024-04-22 16:36:15.714995'), ('花蓮縣', '46', '2024-04-22 16:36:15.724138'), ('臺東縣', '23', '2024-04-22 16:36:15.733082'), ('金門縣', '6', '2024-04-22 16:36:15.741091'), ('連江

In [10]:
# 3. 接下來請將昨天計算的「各地區的剩餘口罩」存入資料庫的 `masks` 表格，欄位包含「地區」、「剩餘口罩數量」、「新增時間」，在同一個 Repo、Colab 編輯完成後再 Commit 紀錄成一個版本：
import sqlite3
import datetime
import requests
import json

# 連接至 SQLite 數據庫
conn = sqlite3.connect('mask.db')
c = conn.cursor()

# 創建新表格並清空原有數據
c.execute('''
CREATE TABLE IF NOT EXISTS masks (
    city TEXT,
    counts INTEGER,
    createAt DATETIME
)''')
c.execute('DELETE FROM masks')
conn.commit()

# 下載數據
# 計算各縣市剩餘成人口罩數量
mask_count = {}
for d in data['features']:
    county = d['properties']['county']
    number = int(d['properties']['mask_adult'])
    if county not in mask_count:
        mask_count[county] = number
    else:
        mask_count[county] += number

# 將數據插入數據庫
for city, counts in mask_count.items():
    t = datetime.datetime.now()
    # 使用參數化查詢來增強安全性
    c.execute("INSERT INTO masks (city, counts, createAt) VALUES (?, ?, ?)", (city, counts, t))
    conn.commit()

# 查詢並打印所有記錄
c.execute("SELECT * FROM masks")
print(c.fetchall())

# 提交事務並關閉連接
conn.commit()
conn.close()



[('臺北市', '322490', '2024-04-22 16:48:34.468891'), ('', '69210', '2024-04-22 16:48:34.478259'), ('高雄市', '424170', '2024-04-22 16:48:34.486894'), ('臺中市', '536710', '2024-04-22 16:48:34.495121'), ('臺南市', '393900', '2024-04-22 16:48:34.502508'), ('基隆市', '80100', '2024-04-22 16:48:34.509563'), ('新竹市', '45630', '2024-04-22 16:48:34.516922'), ('嘉義市', '107840', '2024-04-22 16:48:34.524287'), ('新北市', '670270', '2024-04-22 16:48:34.531748'), ('桃園市', '331940', '2024-04-22 16:48:34.539328'), ('新竹縣', '51350', '2024-04-22 16:48:34.546616'), ('宜蘭縣', '106900', '2024-04-22 16:48:34.554542'), ('苗栗縣', '131080', '2024-04-22 16:48:34.562587'), ('彰化縣', '219400', '2024-04-22 16:48:34.571134'), ('南投縣', '91240', '2024-04-22 16:48:34.578888'), ('雲林縣', '196990', '2024-04-22 16:48:34.586290'), ('嘉義縣', '113180', '2024-04-22 16:48:34.594085'), ('屏東縣', '184250', '2024-04-22 16:48:34.600859'), ('澎湖縣', '12600', '2024-04-22 16:48:34.608687'), ('花蓮縣', '57330', '2024-04-22 16:48:34.616313'), ('臺東縣', '28510', '2024-04-22 